In [3]:
import re
import collections
BlockArgs = collections.namedtuple('BlockArgs', [
    'kernel_size', 'num_repeat', 'input_filters', 'output_filters',
    'expand_ratio', 'id_skip', 'stride', 'se_ratio'])

# Change namedtuple defaults
# GlobalParams.__new__.__defaults__ = (None,) * len(GlobalParams._fields)
BlockArgs.__new__.__defaults__ = (None,) * len(BlockArgs._fields)

In [4]:
BlockArgs

__main__.BlockArgs

In [77]:
def _decode_block_string(block_string):
    """ Gets a block through a string notation of arguments. """
    assert isinstance(block_string, str)

    ops = block_string.split('_')
    options = {}
    for op in ops:
        splits = re.split(r'(\d.*)', op)
        if len(splits) >= 2:
            key, value = splits[:2]
            options[key] = value

    # Check stride
    # assert (('s' in options and len(options['s']) == 1) or
    #         (len(options['s']) == 2 and options['s'][0] == options['s'][1]))
    if len(options['k'])>=2:
        kernel_size = (int(options['k'][:-1]), int(options['k'][-1]))
    else:
        kernel_size = (int(options['k']),int(options['k']))

    if len(options['s'])>=2:
        stride = (int(options['s'][:-1]), int(options['s'][-1]))
    else:
        stride = (int(options['s']),int(options['s']))

    return BlockArgs(
        kernel_size=kernel_size,
        num_repeat=int(options['r']),
        input_filters=int(options['i']),
        output_filters=int(options['o']),
        expand_ratio=int(options['e']),
        id_skip=('noskip' not in block_string),
        se_ratio=float(options['se']) if 'se' in options else None,
        stride=stride)

In [83]:
_decode_block_string('r1_k223_s224_e1_i32_o16_se0.25')

BlockArgs(kernel_size=(22, 3), num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, stride=(22, 4), se_ratio=0.25)

In [45]:
stride = _decode_block_string('r1_k3_s21_e1_i32_o16_se0.25').stride
print(stride)
len(stride)

# stride = stride if len(stride) == 2 else [stride[0]] * 2

[21]


1

In [30]:
class Conv2dStaticSamePadding(nn.Conv2d):
    """ 2D Convolutions like TensorFlow, for a fixed image size"""

    def __init__(self, in_channels, out_channels, kernel_size, image_size=None, **kwargs):
        super().__init__(in_channels, out_channels, kernel_size, **kwargs)
        self.stride = self.stride if len(self.stride) == 2 else [self.stride[0]] * 2

        # Calculate padding based on image size and save it
        assert image_size is not None
        ih, iw = image_size if type(image_size) == list else [image_size, image_size]
        kh, kw = self.weight.size()[-2:]
        sh, sw = self.stride
        oh, ow = math.ceil(ih / sh), math.ceil(iw / sw)
        pad_h = max((oh - 1) * self.stride[0] + (kh - 1) * self.dilation[0] + 1 - ih, 0)
        pad_w = max((ow - 1) * self.stride[1] + (kw - 1) * self.dilation[1] + 1 - iw, 0)
        if pad_h > 0 or pad_w > 0:
            self.static_padding = nn.ZeroPad2d((pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2))
        else:
            self.static_padding = Identity()


[11, 11]